In [1]:
import time
import numpy as np
import cv2
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy import signal
import os, errno
import glob

In [2]:
## get first and last centroids to 1) Draw tracks 2) do calculate radial distance from flower center

In [5]:
Centroidfiles_direc = r'G:\My Drive\Mahad\Light Level Experiments\Data Output from Analysis/motionAnalysis_Final/Centroid/Centroids_Relevant_v4/'
centroid_files = glob.glob(Centroidfiles_direc + 'L*.csv')

In [6]:
centroid_files[0]

'G:\\My Drive\\Mahad\\Light Level Experiments\\Data Output from Analysis/motionAnalysis_Final/Centroid/Centroids_Relevant_v4\\L50_c-3_m9_firstVisit.csv'

In [7]:
treatments = ['L0.1' , 'L50']
visits = ['first', 'last']

In [8]:
for v in visits:
    relevantfiles = [f for f in centroid_files if v in f]
    for l in treatments:
        subsetFiles = [f for f in relevantfiles if l in f]
        print('number of moths in %s and %s is %d' %(v, l, len(subsetFiles)))

number of moths in first and L0.1 is 29
number of moths in first and L50 is 26
number of moths in last and L0.1 is 29
number of moths in last and L50 is 26


In [9]:
df = pd.DataFrame(data = None, columns = ['lightLevel', 'visitNum', 'flower', 'mothID', 'x', 'y'])

lux = []
flower = []
moth = []
visitNum = []
centroids_x = []
centroids_y = []

for f in centroid_files:
    
    cent = pd.read_csv(f)
    x_pos = cent.X_position
    y_pos = cent.Y_position
    
    n = len(x_pos)
    l, c, m, v = os.path.basename(f).split('_')
    padded_l = [l]*n
    padded_c = [c]*n
    padded_m = [m]*n
    padded_v = [v]*n
    
    lux.extend(padded_l)
    flower.extend(padded_c)
    moth.extend(padded_m)
    visitNum.extend(padded_v)
    centroids_x.extend(x_pos)
    centroids_y.extend(y_pos)

df = pd.DataFrame({'lightLevel': lux
                   , 'visitNum': visitNum
                   , 'flower' : flower
                   , 'mothID': moth
                   , 'x': centroids_x
                   , 'y': centroids_y
                  })

df['visitNum'] = df['visitNum'].replace({'lastVisit.csv':'last', 'firstVisit.csv':'first'})

In [10]:
for v, subdf in df.groupby('visitNum'):
    for l, subsubdf in subdf.groupby('lightLevel'):
        totalMoths = subsubdf.mothID.unique()
        print('number of moths in %s and %s is %d' %(v, l, len(totalMoths)))

number of moths in first and L0.1 is 29
number of moths in first and L50 is 26
number of moths in last and L0.1 is 29
number of moths in last and L50 is 26


In [11]:
df.head()

,lightLevel,visitNum,flower,mothID,x,y
0,L50,first,c-3,m9,NaN,NaN
1,L50,first,c-3,m9,NaN,NaN
2,L50,first,c-3,m9,NaN,NaN
3,L50,first,c-3,m9,3.108108,14.256757
4,L50,first,c-3,m9,9.222973,15.236486


In [12]:
# add the distance measurement to all points

In [15]:
circ_parameters = pd.read_csv('G:\My Drive\Mahad\Light Level Experiments\Data Output from Analysis\motionAnalysis_Final\Centroid\circle_parameters.csv')
circ_parameters.head()

,Unnamed: 0,name,circ_x,circ_y,circ_radii
0,0,L0.1_c-3_m27,311,233,74
1,1,L0.1_c-3_m25,309,232,74
2,2,L0.1_c-3_m24,310,233,74
3,3,L0.1_c-3_m23,310,233,74
4,4,L0.1_c-3_m22,310,232,74


In [16]:
df['name'] = df['lightLevel'] + '_' + df['flower'] + '_' + df['mothID']
df["flower_x"] = np.nan
df["flower_y"] = np.nan

In [17]:
for n,subdf in df.groupby('name'):
    r = circ_parameters.loc[circ_parameters.name == n, 'circ_radii'].values
    circ_x =  circ_parameters.loc[circ_parameters.name == n, 'circ_x'].values*2.5/r
    circ_y =  circ_parameters.loc[circ_parameters.name == n, 'circ_y'].values*2.5/r
    
    l = len(subdf)
    f_x = [circ_x]*l
    f_y = [circ_y]*l
    
    df.loc[df.name == n, 'flower_x'] = f_x
    df.loc[df.name == n, 'flower_y'] = f_y
#     df['f_x'] = circ_parameters.loc[circ_parameters.name == df['name'], 'circ_x']

In [18]:
df.head()

,lightLevel,visitNum,flower,mothID,x,y,name,flower_x,flower_y
0,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622
1,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622
2,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622
3,L50,first,c-3,m9,3.108108,14.256757,L50_c-3_m9,10.472973,7.871622
4,L50,first,c-3,m9,9.222973,15.236486,L50_c-3_m9,10.472973,7.871622


In [19]:
df['cent_x'] = df['flower_x'] - df['x']
df['cent_y'] = df['flower_y'] - df['y']

In [20]:
df['distance'] = df.loc[:,['cent_x', 'cent_y']].apply(np.linalg.norm, axis=1, ord = 2)

In [21]:
df.head()

,lightLevel,visitNum,flower,mothID,x,y,name,flower_x,flower_y,cent_x,cent_y,distance
0,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
1,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
2,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
3,L50,first,c-3,m9,3.108108,14.256757,L50_c-3_m9,10.472973,7.871622,7.364865,-6.385135,9.747368
4,L50,first,c-3,m9,9.222973,15.236486,L50_c-3_m9,10.472973,7.871622,1.250000,-7.364865,7.470190


In [ ]:
# df.to_csv('./dataFolders/Output/KinematicAnalysis_v5/CentroidData.csv')

In [ ]:
# missing link - can't find the code that generated the file names CentroidDataAndDistance.csv. 
# so I am going to recompute and export it to make sure the values are all the same and as expected!

In [22]:
# transform them back into pixel

In [28]:
circle = circ_parameters.copy()

In [23]:
def GetCircParams(name):
    r = circle[circle.name == name].circ_radii.values[0]
    x = circle[circle.name == name].circ_x.values[0]
    y = circle[circle.name == name].circ_y.values[0]
    return(r, x, y)

In [24]:
def convertToPixel(x,y,r):
    new_x = x*r/2.5
    new_y = y*r/2.5
    return(new_x, new_y)

In [26]:
centroid = df.copy()

In [30]:
#create a pd with all x and y centroids in pixel space

mothname = []
visit = []
flower_r = []
flower_x = []
flower_y = []
centroid_x = []
centroid_y = []
p_x = []
p_y = []

for name, new_df in centroid.groupby('name'):
    x = new_df.x
    y = new_df.y
    
    f_r, f_x, f_y = GetCircParams(name)
    # convert back to pixel space
    pixel_x, pixel_y = convertToPixel(x,y,f_r)
       
    r = [f_r]*len(pixel_x)
    circ_x = [f_x]*len(pixel_x)
    circ_y = [f_y]*len(pixel_x)
    n = [name]*len(pixel_x)
    v = new_df.visitNum
    
    mothname.extend(n)
    flower_r.extend(r)
    flower_x.extend(circ_x)
    flower_y.extend(circ_y)
    centroid_x.extend(x)
    centroid_y.extend(y)
    p_x.extend(pixel_x)
    p_y.extend(pixel_y)
    visit.extend(v)
    
uber_df = pd.DataFrame({'name' : mothname,
                        'visit' : visit,
                       'flower_r': flower_r,
                       'flower_x': flower_x,
                       'flower_y': flower_y,
                        'x':centroid_x,
                        'y':centroid_y,
                        'x_pixel': p_x,
                        'y_pixel': p_y})

In [31]:
uber_df.head()

,name,visit,flower_r,flower_x,flower_y,x,y,x_pixel,y_pixel
0,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
1,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
2,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
3,L0.1_c-3_m10,last,74,310,233,0.810811,2.297297,24.0,68.0
4,L0.1_c-3_m10,last,74,310,233,1.013514,2.770270,30.0,82.0


In [32]:
# compare this dataframe with the saved centroid and distance data

In [64]:
compare_df = pd.read_csv('./dataFolders/Output/KinematicAnalysis_v5/CentroidAndDistanceData.csv')

In [65]:
compare_df.head()

,Unnamed: 0,Unnamed: 0.1,name,visit,flower_r,flower_x,flower_y,x,y,x_pixel,y_pixel,x_smoothened,y_smoothened,cent_x,cent_y,distance,distanceincms
0,0,0,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,L0.1_c-3_m10,last,74,310,233,0.810811,2.297297,24.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,L0.1_c-3_m10,last,74,310,233,1.013514,2.770270,30.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
compare_df.index = compare_df['Unnamed: 0.1']

In [67]:
compare_df = compare_df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1',
                          'x_smoothened', 'y_smoothened',
                          'cent_x', 'cent_y', 
                          'distance',
                          'distanceincms'])

In [41]:
uber_df.head()

,name,visit,flower_r,flower_x,flower_y,x,y,x_pixel,y_pixel
0,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
1,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
2,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
3,L0.1_c-3_m10,last,74,310,233,0.810811,2.297297,24.0,68.0
4,L0.1_c-3_m10,last,74,310,233,1.013514,2.770270,30.0,82.0


In [68]:
uber_df.equals(compare_df)

False

In [40]:
len(uber_df), len(compare_df)

(96380, 96380)

In [79]:
compare = pd.DataFrame()

for c in uber_df.columns:
    compare[c] = np.where(uber_df[c] == compare_df[c], 'True', 'False')  #create new column in df1 to check if prices match
    print(compare[compare[c] == False])

Empty DataFrame
Columns: [name]
Index: []
Empty DataFrame
Columns: [name, visit]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x, flower_y]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x, flower_y, x]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x, flower_y, x, y]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x, flower_y, x, y, x_pixel]
Index: []
Empty DataFrame
Columns: [name, visit, flower_r, flower_x, flower_y, x, y, x_pixel, y_pixel]
Index: []


In [75]:
# none of the columns seem to be different!

In [76]:
uber_df.to_csv('./dataFolders/Output/KinematicAnalysis_Final/CentroidData.csv')